In [1]:
import pandas as pd

In [3]:
df_de = pd.read_json('./train_DE_processado.json')
df_es = pd.read_json('./train_ES_processado.json')
df_fr = pd.read_json('./train_FR_processado.json')
df_it = pd.read_json('./train_IT_processado.json')
df_ja = pd.read_json('./train_JA_processado.json')

In [4]:
from deeppavlov import build_model

ner_model_en = build_model('ner_ontonotes_bert', download=True, install=True)
ner_model_mult = build_model('ner_ontonotes_bert_mult', download=True, install=True)

Ignoring transformers: markers 'python_version < "3.8"' don't match your environment


2024-12-11 20:49:23.548 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/v1/ner/ner_ontonotes_bert_torch_crf.tar.gz download because of matching hashes
/home/lucas/projects/master-degree/llm-discipline/homework-01-ner/env/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of

Ignoring transformers: markers 'python_version < "3.8"' don't match your environment


2024-12-11 20:49:46.736 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/v1/ner/ner_ontonotes_bert_mult_torch_crf.tar.gz download because of matching hashes
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSeq

In [5]:
import html

def decode_html_entities(text):
    return html.unescape(text)

def get_ner(df, output_csv):
    df['source_text'] = df['source_text'].apply(decode_html_entities)
    df['target_text'] = df['target_text'].apply(decode_html_entities)
    
    df['source_text_ner'] = None
    df['target_text_ner'] = None
    df['source_text_ner_tokens'] = None  # Initialize token columns
    df['target_text_ner_tokens'] = None

    total_rows = len(df)
    print(f"Processing {total_rows} rows...")

    for index, row in df.iterrows():
        source_text = row['source_text']
        target_text = row['target_text']

        try:
            if pd.notna(source_text):
                response = ner_model_en([source_text])
                if response and len(response[0]) > 0:
                    source_ner = response[1][0]
                    source_ner_tokens = response[0][0]
                    df.at[index, 'source_text_ner'] = source_ner
                    df.at[index, 'source_text_ner_tokens'] = source_ner_tokens

            if pd.notna(target_text):
                response = ner_model_mult([target_text])
                if response and len(response[0]) > 0:
                    target_ner = response[1][0]
                    target_ner_tokens = response[0][0]
                    df.at[index, 'target_text_ner'] = target_ner
                    df.at[index, 'target_text_ner_tokens'] = target_ner_tokens
        except Exception as e:
            print(f"Error processing row {index}: {e}")
            df.at[index, 'source_text_ner'] = None
            df.at[index, 'target_text_ner'] = None
            df.at[index, 'source_text_ner_tokens'] = None
            df.at[index, 'target_text_ner_tokens'] = None

        if (index + 1) % 10 == 0 or (index + 1) == total_rows:
            print(f"Processed {index + 1}/{total_rows} rows...")

    df.to_csv(output_csv, index=False)

In [6]:
get_ner(df_de, 'semeval_de_ner.csv')

Processing 4087 rows...


/home/lucas/projects/master-degree/llm-discipline/homework-01-ner/env/lib/python3.10/site-packages/torchcrf/__init__.py:305: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at ../aten/src/ATen/native/TensorCompare.cpp:413.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


Processed 10/4087 rows...
Processed 20/4087 rows...
Processed 30/4087 rows...
Processed 40/4087 rows...
Processed 50/4087 rows...
Processed 60/4087 rows...
Processed 70/4087 rows...
Processed 80/4087 rows...
Processed 90/4087 rows...
Processed 100/4087 rows...
Processed 110/4087 rows...
Processed 120/4087 rows...
Processed 130/4087 rows...
Processed 140/4087 rows...
Processed 150/4087 rows...
Processed 160/4087 rows...
Processed 170/4087 rows...
Processed 180/4087 rows...
Processed 190/4087 rows...
Processed 200/4087 rows...
Processed 210/4087 rows...
Processed 220/4087 rows...
Processed 230/4087 rows...
Processed 240/4087 rows...
Processed 250/4087 rows...
Processed 260/4087 rows...
Processed 270/4087 rows...
Processed 280/4087 rows...
Processed 290/4087 rows...
Processed 300/4087 rows...
Processed 310/4087 rows...
Processed 320/4087 rows...
Processed 330/4087 rows...
Processed 340/4087 rows...
Processed 350/4087 rows...
Processed 360/4087 rows...
Processed 370/4087 rows...
Processed 

In [7]:
get_ner(df_es, 'semeval_es_ner.csv')

Processing 5160 rows...
Processed 10/5160 rows...
Processed 20/5160 rows...
Processed 30/5160 rows...
Processed 40/5160 rows...
Processed 50/5160 rows...
Processed 60/5160 rows...
Processed 70/5160 rows...
Processed 80/5160 rows...
Processed 90/5160 rows...
Processed 100/5160 rows...
Processed 110/5160 rows...
Processed 120/5160 rows...
Processed 130/5160 rows...
Processed 140/5160 rows...
Processed 150/5160 rows...
Processed 160/5160 rows...
Processed 170/5160 rows...
Processed 180/5160 rows...
Processed 190/5160 rows...
Processed 200/5160 rows...
Processed 210/5160 rows...
Processed 220/5160 rows...
Processed 230/5160 rows...
Processed 240/5160 rows...
Processed 250/5160 rows...
Processed 260/5160 rows...
Processed 270/5160 rows...
Processed 280/5160 rows...
Processed 290/5160 rows...
Processed 300/5160 rows...
Processed 310/5160 rows...
Processed 320/5160 rows...
Processed 330/5160 rows...
Processed 340/5160 rows...
Processed 350/5160 rows...
Processed 360/5160 rows...
Processed 370

In [8]:
get_ner(df_fr, 'semeval_fr_ner.csv')

Processing 5531 rows...
Processed 10/5531 rows...
Processed 20/5531 rows...
Processed 30/5531 rows...
Processed 40/5531 rows...
Processed 50/5531 rows...
Processed 60/5531 rows...
Processed 70/5531 rows...
Processed 80/5531 rows...
Processed 90/5531 rows...
Processed 100/5531 rows...
Processed 110/5531 rows...
Processed 120/5531 rows...
Processed 130/5531 rows...
Processed 140/5531 rows...
Processed 150/5531 rows...
Processed 160/5531 rows...
Processed 170/5531 rows...
Processed 180/5531 rows...
Processed 190/5531 rows...
Processed 200/5531 rows...
Processed 210/5531 rows...
Processed 220/5531 rows...
Processed 230/5531 rows...
Processed 240/5531 rows...
Processed 250/5531 rows...
Processed 260/5531 rows...
Processed 270/5531 rows...
Processed 280/5531 rows...
Processed 290/5531 rows...
Processed 300/5531 rows...
Processed 310/5531 rows...
Processed 320/5531 rows...
Processed 330/5531 rows...
Processed 340/5531 rows...
Processed 350/5531 rows...
Processed 360/5531 rows...
Processed 370

In [9]:
get_ner(df_it, 'semeval_it_ner.csv')

Processing 3739 rows...
Processed 10/3739 rows...
Processed 20/3739 rows...
Processed 30/3739 rows...
Processed 40/3739 rows...
Processed 50/3739 rows...
Processed 60/3739 rows...
Processed 70/3739 rows...
Processed 80/3739 rows...
Processed 90/3739 rows...
Processed 100/3739 rows...
Processed 110/3739 rows...
Processed 120/3739 rows...
Processed 130/3739 rows...
Processed 140/3739 rows...
Processed 150/3739 rows...
Processed 160/3739 rows...
Processed 170/3739 rows...
Processed 180/3739 rows...
Processed 190/3739 rows...
Processed 200/3739 rows...
Processed 210/3739 rows...
Processed 220/3739 rows...
Processed 230/3739 rows...
Processed 240/3739 rows...
Processed 250/3739 rows...
Processed 260/3739 rows...
Processed 270/3739 rows...
Processed 280/3739 rows...
Processed 290/3739 rows...
Processed 300/3739 rows...
Processed 310/3739 rows...
Processed 320/3739 rows...
Processed 330/3739 rows...
Processed 340/3739 rows...
Processed 350/3739 rows...
Processed 360/3739 rows...
Processed 370

In [10]:
get_ner(df_ja, 'semeval_ja_ner.csv')

Processing 7225 rows...
Processed 10/7225 rows...
Processed 20/7225 rows...
Processed 30/7225 rows...
Processed 40/7225 rows...
Processed 50/7225 rows...
Processed 60/7225 rows...
Processed 70/7225 rows...
Processed 80/7225 rows...
Processed 90/7225 rows...
Processed 100/7225 rows...
Processed 110/7225 rows...
Processed 120/7225 rows...
Processed 130/7225 rows...
Processed 140/7225 rows...
Processed 150/7225 rows...
Processed 160/7225 rows...
Processed 170/7225 rows...
Processed 180/7225 rows...
Processed 190/7225 rows...
Processed 200/7225 rows...
Processed 210/7225 rows...
Processed 220/7225 rows...
Processed 230/7225 rows...
Processed 240/7225 rows...
Processed 250/7225 rows...
Processed 260/7225 rows...
Processed 270/7225 rows...
Processed 280/7225 rows...
Processed 290/7225 rows...
Processed 300/7225 rows...
Processed 310/7225 rows...
Processed 320/7225 rows...
Processed 330/7225 rows...
Processed 340/7225 rows...
Processed 350/7225 rows...
Processed 360/7225 rows...
Processed 370